In [3]:
import pandas as pd
import plotly.express as px
import numpy as np

import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_dark"

races = pd.read_csv('data/races.csv')
races["EventDate"] = pd.to_datetime(races["EventDate"])
races["EventDate"] = races["EventDate"].dt.date
races['Year'] = races['Year'].astype(str)

chosen_year = '2021'
chosen_driver = 'De Vries'
chosen_gp = 'Bahrain Grand Prix'

year_df = races[races.Year == chosen_year]
driver_df = races.loc[races['LastName'] == chosen_driver].sort_values(by=['EventDate'])
gp_df = races.loc[races['GP'] == chosen_gp]
driver_year_df = driver_df.loc[driver_df['Year'] == chosen_year].sort_values(by=['EventDate'])

# BAR GRAPHS

In [ ]:
season_points = px.histogram(driver_df, x='Year', y='Points', title=f'Points by Season - {chosen_driver}', labels = {'Points':'Points', 'Year':'Season'}, color='TeamName',text_auto=True, opacity=0.9)
season_points.show()

In [ ]:
driver_yr_summary = driver_df.groupby('Year').agg(
    TotalPoints = pd.NamedAgg(column="Points", aggfunc=sum),
    TeamName = pd.NamedAgg(column="TeamName", aggfunc=max),
    TotalRaces = pd.NamedAgg(column="Counter", aggfunc=sum)
)
driver_yr_summary["AveragePoints"] = driver_yr_summary.TotalPoints / driver_yr_summary.TotalRaces
driver_yr_summary = driver_yr_summary.reset_index()

avg_season_points = px.histogram(driver_yr_summary, x='Year', y='AveragePoints', title=f'Average Points by Season - {chosen_driver}', labels = {'Points':'Points', 'Year':'Season'}, color = "TeamName", text_auto=True, opacity=0.9)
avg_season_points.show()

# PIES

In [ ]:
driver_success = px.pie(driver_df, values='Counter', names='ResultType', title=f'Race Results for Career - {chosen_driver}')
driver_success.show()

In [ ]:
driver_df_dnf = driver_df.query("Status != 'Finished'").query("Status != '+1 Lap'").query("Status != '+2 Laps'")
dnf_type_pie = px.pie(data_frame = driver_df_dnf, values ='Counter', names = 'Status', title="Most frequent cause of DNF" )
dnf_type_pie.show()

# DATA TABLES

In [ ]:

season_summary = driver_year_df[['RoundNumber', 'EventDate', 'GP', 'EventFormat', 'Location', 'Country', 'QualiStatus', 'GridPosition','ResultType', 'Status','Position', 'Points']]
season_summary

# GAUGE

In [41]:
category_races[category_races[chart_category]== "Finished"]["Counter"][0]

1

In [ ]:
chart_category = "Status"
category_races = driver_df[[chart_category, "Counter"]].groupby(chart_category).sum().reset_index()

one_lap_added = (category_races[category_races[chart_category]== "+1 Lap"]["Counter"].values[0])
two_lap_added = (category_races[category_races[chart_category]== "+2 Laps"]["Counter"].values[0])
sum_completed = (category_races[category_races[chart_category]== "Finished"]["Counter"].values[0])
data_races_completed = ((sum_completed+one_lap_added+two_lap_added)/(category_races["Counter"].sum()))
pct_completed = round(data_races_completed*100, ndigits=2)

fig_races_completed = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = pct_completed,
    number = {"suffix": "%"},
    title = {'text': "% Races Completed"},
    domain = {'x': [0,1], 'y': [0,1]},
    gauge = {'axis': {'range': [0, 100]}}
))

fig_races_completed.show()

#_______________

chart_category = "ResultType"
category_races = driver_df[[chart_category, "Counter"]].groupby(chart_category).sum().reset_index()

data_races_in_points = ((category_races[category_races[chart_category]!= "NoPoints"]["Counter"].values[0])/(category_races["Counter"].sum()))
pct_points = round(data_races_in_points*100, ndigits=2)

fig_races_in_points = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = pct_points,
    number = {"suffix": "%"},
    title = {'text': "% Races in Points"},
    domain = {'x': [0,1], 'y': [0,1]},
    gauge = {'axis': {'range': [0, 100]}}
))

fig_races_in_points.show()

# KPI CARDS

In [ ]:
fig_season_points = go.Figure(go.Indicator(
    mode = "number",
    value = driver_year_df["Points"].sum(),
    title = {'text': f"Total Points for {chosen_year} season"},
    domain = {'x': [0,1], 'y': [0,1]}
))
fig_season_points.show()

In [ ]:
fig_career_points = go.Figure(go.Indicator(
    mode = "number",
    value = driver_df["Points"].sum(),
    title = {'text': "Total Points 2018-2022"},
    domain = {'x': [0,1], 'y': [0,1]}
))
fig_career_points.show()

In [ ]:
fig_podiums = go.Figure(go.Indicator(
    mode = "number",
    value = driver_df.query("ResultType == 'Podium' or ResultType == 'Pole' ")["Counter"].sum(),
    title = {'text': "Number of Podiums 2018-2022"},
    domain = {'x': [0,1], 'y': [0,1]}
))
fig_podiums.show()

In [ ]:
fig_highest_position = go.Figure(go.Indicator(
    mode = "number",
    value = driver_df["Position"].min(),
    title = {'text': "Highest Race Position 2018-2022"},
    domain = {'x': [0,1], 'y': [0,1]}
))
fig_highest_position.show()

In [ ]:
fig_race_wins = go.Figure(go.Indicator(
    mode = "number",
    value = driver_df.query("Position == 1").sum()["Counter"],
    title = {'text': "Number of Wins 2018-2022"},
    domain = {'x': [0,1], 'y': [0,1]}
))
fig_race_wins.show()

# SCATTER

In [ ]:

fig_pos_scatter = px.scatter(
    data_frame = driver_df,
    x = "GridPosition",
    y = "Position",
    range_x = [0,20],
    range_y = [0,20],
    color = "Year",
    trendline="ols",
    trendline_scope= "overall"
)
fig_pos_scatter.show()

# AREA AND LINE

In [ ]:
# add option to choose x = position, gridposition, points

all_options = ["Points", "Position", "GridPosition"]

fig_season_progression = px.area(data_frame=driver_year_df, x="EventDate", y=["Position", "GridPosition"], range_y = [0,20], title=f"Overview of {chosen_year} Season")
fig_season_progression.show()

fig_overall_progression = px.line(data_frame=driver_df, x="EventDate", y=["Position", "GridPosition"], range_y = [0,20],  title="Overview 2018-2022")
fig_overall_progression.show()

In [ ]:
twitter_feed = html.Iframe(srcDoc=''' <a class="twitter-timeline" data-theme="dark" href="https://twitter.com/Carlossainz55"> Tweets by Carlos Sainz </a> 
            <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>''', height=800, width=300, id = "twitter")


In [1]:
chosen_driver = "Carlos Sainz"
import pandas as pd
drivers = pd.read_csv('data/drivers.csv')

In [4]:
sorted(drivers["FlagURL"].unique())


['au',
 'be',
 'br',
 'ca',
 'cn',
 'de',
 'dk',
 'es',
 'fi',
 'fr',
 'gb',
 'it',
 'jp',
 'mc',
 'mx',
 'nl',
 'nz',
 'pl',
 'ru',
 'se',
 'th']

In [23]:
twitter_url

'"https://twitter.com/Carlossainz55"'

In [27]:
twitter_link

'<a class="twitter-timeline" data-theme="dark" href="https://twitter.com/Carlossainz55"> Tweets by Carlos Sainz </a> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>'